In [13]:
import numpy as np

class Node:
    def __init__(self,index, name, mean, std):
        self.index = index
        self.name = name
        self.prior_mean = np.copy(mean)
        self.prior_std = np.copy(std)
        self.posterior_mean=0
        self.posterior_std=0
        self.value = None
        self.parents = []
        self.children = []        

    def add_parent(self, parent):
        self.parents.append(parent)        

    def add_child(self, child):
        self.children.append(child)  

In [14]:
import numpy as np
class BayesNet:
    def __init__(self,num_nodes):
        # num_nodes = number of nodes (int)
        self.num_nodes=num_nodes
        
        #dictionary of names and index of nodes
        self.names_index={}
        
        # structure = shows the structre of BN,(matrix) 
        self.structure=np.empty((num_nodes, num_nodes))
        
        self.weights=np.empty((num_nodes, num_nodes))
        
        # the parameters of nodes, 
        self.params=[]
                
        # the bserved node and their values and array of tuple (node, value)
        #self.observed_nodes=[]
        
        # an array that determines the order of nodes for elimination
        self.orders=[]
        
        #set Nodes
        self.nodes=[]
        for i in range(num_nodes):
            self.nodes.append(Node(i,"", mean=[], std=[]))
    
    #Set names of all nodes
    def set_node_names(self,node_names): 
        self.names_index={}
        for i in range(self.num_nodes):
            self.nodes[i].name = node_names[i] 
            self.names_index[node_names[i]]=i
        return
    
    # Set prior destribution of all nodes
    def set_priors(self, node_prior):
        for i in range(self.num_nodes):
            self.nodes[i].prior_mean =np.copy(node_prior[i]['mean'])  
            self.nodes[i].prior_std=np.copy(node_prior[i]['std'])
        return 
    
    # Set prior destribution of node i
    # the prior of nodes,  it is an array of ([mu1,...,muk],[sgma1, ..., sigmak]) with size num_nodes for a Gaussian mixture model
    def set_prior(self,node_prior,i):
        self.nodes[i].mean =np.copy(node_prior['mean'])
        self.nodes[i].std=np.copy(node_prior['std'])
        return
    
    # Set values of nodes with nodeIndexs (Set all observed_nodes or evidence)
    def set_values(self, assignmens):
        for name,value in assignmens.items():
            self.nodes[self.names_index[name]].value =value
        return
    
    
    def set_structure(self, struct):
        self.structure=np.copy(struct)  
        
        # Establish parent-child relationships based on the adjacency matrix
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if struct[i][j] == 1:
                    self.nodes[i].add_child(self.nodes[j])
                    self.nodes[j].add_parent(self.nodes[i])        
        return
    
    def add_edge(self,parent_name,child_name,weight):
        parent_index=self.names_index[parent_name]
        child_index=self.names_index[child_name]
        self.structure[parent_index][child_index]=1
        self.nodes[parent_index].add_child(self.nodes[child_index])
        self.nodes[child_index].add_parent(self.nodes[parent_index])
        self.params[parent_index][child_index]=weight
        
    #set Parameters or weights of netword
    def set_params(self, parameters): 
        self.params= np.copy(parameters)
        return

    def set_num_node(self, n):
        self.num_node=n
        return
    
    def get_structure(self):
        return self.structure    
           
    def get_num_node(self):
        return self.num_nodes
        
    def get_params(self):
        return self.params
        
    def _get_topological_order(self):
        orders=[]
        nodes=[i for i in range(self.num_nodes)]
        bn_struct=np.copy(struct)
                
        while len(nodes)>0 : 
            
            # Determine the nodes that are in the highest order (zero columns means they have no parent)
            zero_columns = np.all(bn_struct == 0, axis=0)

            # Add nodes to orders                      
            for i in range(len(zero_columns)):
                if zero_columns[i]:
                    orders.append(nodes[i])

            # Remove their columns and rows from bn_struct
            bn_struct=bn_struct[~zero_columns][:, ~zero_columns]
           
            # Remove orderd nodes from nodes
            nodes = [node for node, zero_col in zip(nodes, zero_columns) if not zero_col]
            
        self.orders=orders    
        return orders
    
    def save_to_file(self):
        df = pd.DataFrame(self.params, index=node_names, columns=node_names)
    
    # Calculate additional features (mean and std) for each node
    node_mean = np.mean(matrix, axis=1)
    node_std = np.std(matrix, axis=1)
    
    # Add the additional features as new columns
    df['Mean'] = node_mean
    df['Std'] = node_std
    
    df.to_csv('graph.csv')
    def get_posteriors(self):
        posteriors={}
        for node in self.nodes:
            posteriors[node.name]=[node.posterior_mean,node.posterior_std]
        return posteriors
       
    def perform_inference(self,query_nodes):
        #Get topological order for eleminating variables
        self._get_topological_order()
            
        for i in self.orders:
            node=self.nodes[i] 
            
            # Update posterior for Observed Nodes or Evidences
            if node.value is not None:
                node.posterior_mean=np.array([node.value])                
                node.posterior_std=np.array([0.001])   # Set to 0.001 to avoid devision by zero
                
            
            else:
                # Get mean and std of prior distribution of current node 
                mean, precision = node.prior_mean, (1.0 / (node.prior_std ** 2))
                mean = mean* precision
                
                # Obtained posterior mean and std based on posterior destribution of parents and weights.
                sum_weights=1
                precision_sum = precision
                for i, parent in enumerate(node.parents):
                    weight = self.params[parent.index][node.index]            
                    precision = 1.0 / (parent.posterior_std ** 2)
                    precision_sum = precision_sum + (weight ** 2) * precision
                    mean = mean + (weight ** 2) * parent.posterior_mean * precision         
                node.posterior_mean=mean/precision_sum
                node.posterior_std = np.sqrt(1 / precision_sum)
        
        result=[] 
        for n in query_nodes:
            i=self.names_index[n]
            result.append(np.random.normal(self.nodes[i].posterior_mean[0], self.nodes[i].posterior_std[0]))
        return result


In [12]:
import numpy as np
#Define the node names
node_names=["Flexibility", 
           "UserExperience",
           "Adaptability",
           "Legibility",
           "GroupingByFormat",
           "GroupingByLocation",
           "Grouping",
           "SignificanceOfCodes",
           "TimeGaps",
           "NumberOfUse",
           "Consistency",
           "ImmediateFeedback",
           "ExplicitUserAction",
           "UserControl",
           "ExplicitControl",
           "Prompting",
           "ErrorProtection",
           "ErrorCorrection",
           "QualityOfErrorMessage",
           "ErrorManagement",
           "InformationDensity",
           "MinimalAction",
           "Conciseness",
           "Brevity",
           "Remembering",
           "Learning",
           "CorrectPath",
           "Time",
           "Back",
           "LoopOnTheSamePage",
           "WrongPath",
           "Incomplete"]
# Determine number of nodes, and create empty structure and parameter 
n_nodes=len(node_names)
struct=np.zeros((n_nodes,n_nodes))
params=np.zeros((n_nodes,n_nodes))

# create empty BN for determined nodes
bn=BayesNet(n_nodes)
bn.set_structure(struct)
bn.set_node_names(node_names)
bn.set_params(params)

#Add Edges and Parameters to BN
bn.add_edge("Flexibility","Adaptability",1)
bn.add_edge("UserExperience","Adaptability",1)
bn.add_edge("Adaptability","Remembering",1)
bn.add_edge("Adaptability","Learning",1)
bn.add_edge("Adaptability","Time",1)
bn.add_edge("Adaptability","CorrectPath",1)

bn.add_edge("Legibility","Time",1)

bn.add_edge("GroupingByFormat","Grouping",1)
bn.add_edge("GroupingByLocation","Grouping",1)
bn.add_edge("Grouping","Remembering",1)
bn.add_edge("Grouping","Learning",1)
bn.add_edge("Grouping","Time",1)
bn.add_edge("Grouping","CorrectPath",1)

bn.add_edge("SignificanceOfCodes","Remembering",1)
bn.add_edge("SignificanceOfCodes","CorrectPath",1)

bn.add_edge("TimeGaps","Remembering",1)

bn.add_edge("NumberOfUse","Remembering",1)
bn.add_edge("NumberOfUse","Learning",1)

bn.add_edge("Consistency","Remembering",1)
bn.add_edge("Consistency","Learning",1)
bn.add_edge("Consistency","Time",1)
bn.add_edge("Consistency","CorrectPath",1)

bn.add_edge("ImmediateFeedback","CorrectPath",1)

bn.add_edge("ExplicitUserAction","ExplicitControl",1)
bn.add_edge("UserControl","ExplicitControl",1)
bn.add_edge("ExplicitControl","Learning",1)

bn.add_edge("Prompting","Remembering",1)
bn.add_edge("Prompting","Learning",1)
bn.add_edge("Prompting","Back",1)
bn.add_edge("Prompting","LoopOnTheSamePage",1)
bn.add_edge("Prompting","WrongPath",1)

bn.add_edge("ErrorProtection","ErrorManagement",1)
bn.add_edge("ErrorCorrection","ErrorManagement",1)
bn.add_edge("QualityOfErrorMessage","ErrorManagement",1)
bn.add_edge("ErrorManagement","Incomplete",1)
bn.add_edge("ErrorManagement","WrongPath",1)
bn.add_edge("ErrorManagement","Learning",1)
bn.add_edge("ErrorManagement","Time",1)

bn.add_edge("InformationDensity","Incomplete",1)
bn.add_edge("InformationDensity","WrongPath",1)
bn.add_edge("InformationDensity","Learning",1)

bn.add_edge("MinimalAction","Brevity",1)
bn.add_edge("Conciseness","Brevity",1)
bn.add_edge("Brevity","Learning",1)
bn.add_edge("Brevity","Remembering",1)
bn.add_edge("Brevity","Time",1)
bn.add_edge("Brevity","LoopOnTheSamePage",1)
bn.add_edge("Brevity","Incomplete",1)

bn.add_edge("Remembering","Time",1)
bn.add_edge("Remembering","CorrectPath",1)

bn.add_edge("Learning","Time",1)
bn.add_edge("Learning","CorrectPath",1)





[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Set Prior distributions of all nodes
Legibility: 0, m= 80,100
Grouping: 50
Immediate feedback: 60
Prompting: 40
Remembering: 50
Learning: 50
Time: 0.1 -  5 – 20
Correct Path:45
Back:20
Loop:20
Wrong:10
Incomplete: 5

Information Density:50 , std high
Brevity: 50
Explicit control: 50 std m
Flexibility
User Experience:
Adaptability: 50
Error Protection:
Quality:
Error Correction:
Error Management: 50
Significance of Code: 45
Consistency: 40

prior=[]
for i in range(5):
    prior.append({"mean":m[i], "std":s[i]})
querynodes=["E","D"]
bn.set_priors(prior)

# Set Evidence
evidences={"A":10}
bn.set_values(evidences)

#perform inference to obtain posterior distributions
bn.perform_inference(querynodes)
print(bn.get_posteriors())

In [105]:
struct= [[0,1,1,0,0],
         [0,0,0,1,1],
         [0,0,0,1,0],
         [0,0,0,0,0],
         [0,0,0,0,0]]

params = [[0,5,5,0,0],
         [0,0,0,5,5],
         [0,0,0,5,0],
         [0,0,0,0,0],
         [0,0,0,0,0]]

bn=BayesNet(5)
bn.set_structure(struct)
bn.set_node_names(["A","B","C","D","E"])

prior=[]
for i in range(5):
    m=np.random.rand(1)
    s=np.random.rand(1)
    prior.append({"mean":m, "std":s})


bn.set_priors(prior)
for node in bn.nodes:
    print(node.prior_mean, node.prior_std)
    

    
bn.set_values({"A":10})


bn.set_params(params)
querynodes=["E","D"]
bn.perform_inference(querynodes)
print(bn.get_posteriors())


[0.59715871] [0.11757092]
[0.59778997] [0.42093245]
[0.67851986] [0.92372933]
[0.84174378] [0.43820858]
[0.6720609] [0.42274467]
{'A': [array([10]), array([0.001])], 'B': [array([9.99999788]), array([0.0002])], 'C': [array([9.99999956]), array([0.0002])], 'D': [array([9.99999868]), array([2.82842693e-05])], 'E': [array([9.99999779]), array([3.99999953e-05])]}


In [ ]:
def forward_pass(node):
        #evidence = []
        #for parent in node.parents:
        #    evidence.append(forward_pass(parent))

        mean, precision = node.mean, (1.0 / (node.std ** 2))
        mean = mean* precision
        # update mean and std based on obtained evidence for parents.
        sum_weights=1
        precision_sum = precision
        for i, parent in enumerate(node.parents):
            weight = self.params[parent.index][node.index]            
            precision = 1 / (parent.std ** 2)
            precision_sum = precision_sum + (weight ** 2) * precision
            mean = mean + (weight ** 2) * parent.mean * precision 
        
        mean=mean/precision_sum
        std = np.sqrt(1 / precision_sum)
           
        
        # set random value for the node 
        node.set_value(np.random.normal(mean, np.sqrt(1.0 / std)))

        return node.value

def update_node_with_evidence(self,node, evidence_value):
    node.mean = evidence_value
    node.std = 0.0  # Set standard deviation to 0 for a point distribution
    return node